In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Annahmen:
# - `df_images` ist ein DataFrame mit Bildpfaden und IDs
# - `df_ingredients` ist ein DataFrame mit Zutateninformationen und IDs
# - `your_label_columns` sind die Spalten, die die Zutateninformationen im DataFrame repräsentieren

# Merge beide DataFrames basierend auf den Bild-IDs
merged_df = pd.merge(df_images, df_ingredients, on='image_id')

# Teile die Daten in Trainings- und Testsets auf
train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)

# Erstelle Daten-Generatoren für Bilder
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="image_path",
    y_col=your_label_columns,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='raw'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="image_path",
    y_col=your_label_columns,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='raw'
)

# Definiere die Modellarchitektur
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Kompiliere das Modell
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trainiere das Modell
model.fit(train_generator, epochs=10, validation_data=test_generator)